# Deck

In [1]:
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

In [2]:
class Card:

    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit
        self.value = values[rank]

    def __str__(self):
        return self.rank + " Of " + self.suit
    

In [3]:
test = Card("Two","Hearts")
test.value

2

In [4]:
import random

class Deck:
    def __init__(self):
        self.all_cards = []
        for rank in ranks:
            for suit in suits:
                card_created = Card(rank, suit)
                self.all_cards.append(card_created)

    def shuffle(self):
        random.shuffle(self.all_cards)

In [5]:
class Player():
    def __init__(self, name, balance=0):
        self.own_hand = []
        self.name = name
        self.balance = balance

    def clear_hand(self):
        self.own_hand.clear()

    def add_cards(self, new_cards):
        if isinstance(new_cards, list):
            self.own_hand.extend(new_cards)
        else:
            self.own_hand.append(new_cards)

    def total(self):
        total = 0 
        ace_count = 0
        for card in self.own_hand:
            total += card.value
            if card.rank == "Ace":
                ace_count += 1
        while total > 21 and ace_count > 0:
            total -= 10
            ace_count -= 1
        return total

    def deposit(self, add_money):
        self.balance += add_money
        print(f"Your new balance is {self.balance}")

    def withdraw(self,take_money):
        if 0<take_money <= self.balance:
            self.balance = self.balance - take_money
            print(f"Your new balance is {self.balance}")
            return True
        else:
            print("You don't have enough money to withdraw")
            return False


In [6]:
class Dealer(Player):
    def __init__(self, name):
        super().__init__(name)
        self.own_deck = []

    def load_deck(self, cards):
        if isinstance(cards, list):
            self.own_deck.extend(cards)
        else:
            self.own_deck.append(cards)

    def deal_one(self):
        return self.own_deck.pop(0)  # or pop() if you prefer LIFO

In [7]:
dealer = Dealer("Dealer")
player1 = Player("Player")

new_deck = Deck()
new_deck.shuffle()

dealer.load_deck(new_deck.all_cards)

len(dealer.own_deck)


52

# Logic of the Game

In [8]:
dealer = Dealer("Dealer")
player1 = Player("Player", balance = 1000)

new_deck = Deck()
new_deck.shuffle()

dealer.load_deck(new_deck.all_cards)
stake = ""

game_on = True

while game_on:
    round_over = False
    print(f"Your balance is: {player1.balance}")
    if player1.balance > 0:
        while True:
            stake = input("How much do you want to bet? ")
            if stake.isdigit():
                stake = int(stake)
                if player1.withdraw(stake):
                    break
            else:
                print("That is not a number")
    else:
        print("You don't have enough funds")
        continue

    player1.clear_hand()
    dealer.clear_hand()

    card1 = dealer.deal_one()
    player1.add_cards(card1)

    card2 = dealer.deal_one()
    player1.add_cards(card2)

    card3 = dealer.deal_one()
    dealer.add_cards(card3)

    card4 = dealer.deal_one()
    dealer.add_cards(card4)

    total_player = player1.total()
    total_dealer = card3.value

    print(f"""
    Your cards are:
    {card1}
    {card2}
    Total: {total_player}

    Dealer's hand:
    {card3}
    Hidden
    Total: {total_dealer}
    """)

    while total_player < 21 and not round_over:
        hit = input("Do you want to hit? Y/N ").lower()
        if hit == "y":
            new_card = dealer.deal_one()
            player1.add_cards(new_card)
            print(f"You got: {new_card}")
            total_player = player1.total()
            print(f"Your new total is: {total_player}")
                
            if total_player > 21:
                print("Bust! You went over 21.")
                round_over = True

                      
        elif hit == "n":
            print(f"The hidden card is: {card4}")
            while total_dealer < 17:
                new_card1 = dealer.deal_one()
                dealer.add_cards(new_card1)
                total_dealer = dealer.total()
                print(new_card1)
                print(f"Total: {total_dealer}")

            if total_dealer > 21:
                print("Bust! Dealer went over 21.")
                player1.deposit(stake*2)

            elif total_dealer > total_player:
                print("You lost")
                    
            elif total_player > total_dealer:
                print("You win")
                player1.deposit(stake*2)
                    
            elif total_player == total_dealer:
                print("Its a tie")
                player1.deposit(stake)    
            round_over = True
            
    if total_player == 21 and not round_over:
        print(f"The hidden card is: {card4}")

        while total_dealer < 17:
            new_card1 = dealer.deal_one()
            dealer.add_cards(new_card1)
            total_dealer = dealer.total()
            print(new_card1)
            print(f"Total: {total_dealer}")

        if total_dealer > 21:
            print("Bust! Dealer went over 21.")
            player1.deposit(stake * 2)
        elif total_dealer > total_player:
            print("You lost")
        elif total_player > total_dealer:
            print("You win")
            player1.deposit(stake * 2)
        else:
            print("It's a tie")
            player1.deposit(stake)

        round_over = True
        
    if player1.balance == 0:
        print("You have no money left. Game over.")
        break
                    
    while True:
        replay = input("Replay? Y/N ").lower()
        if replay == "n":
            game_on = False
            break
        elif replay == "y":
            break
        else:
            print("Please choose Y or N")


Your balance is: 1000


How much do you want to bet?  500


Your new balance is 500

    Your cards are:
    Nine Of Spades
    Jack Of Hearts
    Total: 19

    Dealer's hand:
    Seven Of Spades
    Hidden
    Total: 7
    


Do you want to hit? Y/N  n


The hidden card is: Six Of Spades
Three Of Hearts
Total: 16
Four Of Diamonds
Total: 20
You lost


...

Your balance is: 2000


How much do you want to bet?  2000


Your new balance is 0

    Your cards are:
    Nine Of Diamonds
    Eight Of Spades
    Total: 17

    Dealer's hand:
    Four Of Hearts
    Hidden
    Total: 4
    


Do you want to hit? Y/N  n


The hidden card is: Six Of Diamonds
Nine Of Hearts
Total: 19
You lost
You have no money left. Game over.
